In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

## 1. Data retrieval

In [ ]:
DATASETS_DIR = 'datasets/' 
RETRIEVED_DATA = 'raw-data.csv'

def data_retrieval(url):
     
    # Loading data from specific url
    data = pd.read_csv(url)
    
    # Uncovering missing data
    data.replace('?', np.nan, inplace=True)
    data['age'] = data['age'].astype('float')
    data['fare'] = data['fare'].astype('float')
    
    # helper function 1
    def get_first_cabin(row):
        try:
            return row.split()[0]
        except:
            return np.nan
    
    # helper function 2
    def get_title(passenger):
        line = passenger
        if re.search('Mrs', line):
            return 'Mrs'
        elif re.search('Mr', line):
            return 'Mr'
        elif re.search('Miss', line):
            return 'Miss'
        elif re.search('Master', line):
            return 'Master'
        else:
            return 'Other'
    
    # Keep only one cabin | Extract the title from 'name'
    data['cabin'] = data['cabin'].apply(get_first_cabin)
    data['title'] = data['name'].apply(get_title)
    
    # Droping irrelevant columns
    DROP_COLS = ['boat','body','home.dest','ticket','name']
    data.drop(DROP_COLS, 1, inplace=True)
    
    data.to_csv(DATASETS_DIR + RETRIEVED_DATA, index=False)
    
    return print('Data stored in {}'.format(DATASETS_DIR + RETRIEVED_DATA))

In [ ]:
URL = 'https://www.openml.org/data/get_csv/16826755/phpMYEkMl'
data_retrieval(URL)

Data stored in datasets/raw-data.csv


In [ ]:
df = pd.read_csv(DATASETS_DIR + RETRIEVED_DATA)
df.shape

(1309, 10)

In [ ]:
df.sample(5)

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,title
257,1,1,female,35.0,1,0,57.7500,C28,C,Mrs
86,1,1,male,51.0,0,0,26.5500,E17,S,Mr
804,3,0,male,NaN,0,0,7.7500,NaN,Q,Mr
50,1,1,female,58.0,0,1,512.3292,B51,C,Mrs
1304,3,0,female,14.5,1,0,14.4542,NaN,C,Miss


## 2. Train-test split

In [ ]:
SEED_SPLIT = 404

X_train, X_test, y_train, y_test = train_test_split(
                                                        df.drop('survived', axis=1),
                                                        df['survived'],
                                                        test_size=0.2,
                                                        random_state=SEED_SPLIT
                                                   )

TRAIN_DATA_FILE = DATASETS_DIR + 'train.csv'
TEST_DATA_FILE  = DATASETS_DIR + 'test.csv'

X_train.to_csv(TRAIN_DATA_FILE, index=False)
X_test.to_csv(TEST_DATA_FILE, index=False)

X_train.shape, X_test.shape

((1047, 9), (262, 9))

In [ ]:
target = 'survived'
num_vars = [col for col in X_train.columns if X_train[col].dtype != object and col != target]
cat_vars = [col for col in X_train.columns if X_train[col].dtype == object]

In [ ]:
# Validation step
len(num_vars) + len(cat_vars) + 1 == df.shape[1]

True

## 3. Feature engineering

### 3.1. Without persisting information

**Numerical variables**

- Create missing value indicator: only for numeric variables

In [ ]:
def missing_indicator(data, col_name):
    data[col_name+'_nan'] = data[col_name].isnull().astype(int)
    return None

In [ ]:
for var in num_vars:
    missing_indicator(X_train, var)
    missing_indicator(X_test, var)

In [ ]:
X_train.head(2)

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title,pclass_nan,age_nan,sibsp_nan,parch_nan,fare_nan
1162,3,male,NaN,0,0,7.7500,NaN,Q,Mr,0,1,0,0,0
899,3,female,27.0,0,2,11.1333,NaN,S,Mrs,0,0,0,0,0


**Categorical variables**

- Keep only the letter in cabin
- Fill NaN with label "missing"

In [ ]:
def extract_letter_from_cabin(x):
    if type(x)==str:    
        return ''.join(re.findall("[a-zA-Z]+", x))  
    else: 
        return x

X_train['cabin'] = X_train['cabin'].apply(extract_letter_from_cabin)    
X_test['cabin'] = X_test['cabin'].apply(extract_letter_from_cabin)    

In [ ]:
X_train['cabin'].unique(), X_test['cabin'].unique()

(array([nan, 'C', 'E', 'B', 'F', 'D', 'A', 'G'], dtype=object),
 array(['D', nan, 'A', 'B', 'C', 'E', 'G', 'T', 'F'], dtype=object))

In [ ]:
X_train[cat_vars] = X_train[cat_vars].fillna('missing')
X_test[cat_vars]  = X_test[cat_vars].fillna('missing')

### 3.2. With persisting information

**Numerical variables**

- Fill NaN with median

In [ ]:
imp_median = SimpleImputer(strategy='median')
imp_median.fit(X_train[num_vars])

SimpleImputer(strategy='median')

In [ ]:
imp_median.statistics_

array([ 3.    , 28.    ,  0.    ,  0.    , 14.4542])

In [ ]:
X_train[num_vars] = imp_median.transform(X_train[num_vars])
X_test[num_vars]  = imp_median.transform(X_test[num_vars])

**Categorical variables**

- Remove rare labels
- One hot encoding
- Fix one-hot-encoded features not in test set

In [ ]:
def find_rare_labels(data, col, perc):
    data = data.copy()
    tmp = data.groupby(col)[col].count() / data.shape[0]
    return tmp[tmp < perc].index

rare_labels_ = {}
for col in cat_vars:
    rare_labels_[col] = find_rare_labels(X_train, col, 0.05)
    
for col in cat_vars:
    X_train[col] = np.where(X_train[col].isin(rare_labels_[col]), 'Rare', X_train[col])
    X_test[col]  = np.where(X_test[col].isin(rare_labels_[col]), 'Rare', X_test[col])

In [ ]:
X_train[cat_vars[1]].unique()

array(['missing', 'C', 'Rare', 'B'], dtype=object)

In [ ]:
X_train = pd.concat([X_train, pd.get_dummies(X_train[cat_vars], drop_first=True)], 1)
X_test  = pd.concat([X_test, pd.get_dummies(X_test[cat_vars], drop_first=True)], 1)

X_train.drop(cat_vars, 1, inplace=True)
X_test.drop(cat_vars, 1, inplace=True)

In [ ]:
# Validation step
set(X_train.columns).difference(set(X_test.columns))

{'embarked_Rare'}

In [ ]:
for col in list(set(X_train.columns).difference(set(X_test.columns))):
    X_test[col] = 0

**Aligning columns of X_train and X_test**

In [ ]:
ordered_vars = [col for col in X_train.columns]

X_train = X_train[ordered_vars]
X_test  = X_test[ordered_vars]

**Scaling**

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

## 4. Training model

In [ ]:
SEED_MODEL = 404

model = LogisticRegression(C=0.0005, class_weight='balanced', random_state=SEED_MODEL)
model.fit(X_train, y_train)

LogisticRegression(C=0.0005, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=404, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
for s,t in zip(['train','test'],[(X_train, y_train),(X_test,y_test)]):
    x,y = t[0], t[1]
    class_pred = model.predict(x)
    proba_pred = model.predict_proba(x)[:,1]
    print('{} roc-auc : {}'.format(s, roc_auc_score(y, proba_pred)))
    print('{} accuracy: {}'.format(s, accuracy_score(y, class_pred)))
    print()

train roc-auc : 0.8470412710714978
train accuracy: 0.7831900668576887

test roc-auc : 0.8163583073823043
test accuracy: 0.7748091603053435



In [ ]:
tmp = pd.DataFrame(X_test, columns=ordered_vars)
tmp['y_true'] = np.array(y_test)
tmp['y_pred'] = model.predict(X_test)
tmp['proba_pred'] = model.predict_proba(X_test)[:,1]

tmp.head(10)

,pclass,age,sibsp,parch,fare,pclass_nan,age_nan,sibsp_nan,parch_nan,fare_nan,...,cabin_missing,embarked_Q,embarked_Rare,embarked_S,title_Mr,title_Mrs,title_Rare,y_true,y_pred,proba_pred
0,0.0,0.724426,0.000,0.222222,0.221098,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1,0.502177
1,0.5,0.386221,0.125,0.111111,0.051237,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0,0,0.481497
2,1.0,0.223382,0.000,0.000000,0.015379,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1,0.513358
3,0.5,0.423799,0.125,0.000000,0.040989,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0,0,0.481422
4,0.5,0.486430,0.000,0.000000,0.050749,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0,0,0.481452
5,1.0,0.298538,0.000,0.000000,0.013940,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1,0,0.477030
6,0.5,0.160751,0.000,0.111111,0.038061,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1,1,0.514231
7,0.0,0.611691,0.125,0.000000,0.111118,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,0.501921
8,0.0,0.398747,0.000,0.000000,0.148911,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.534687
9,0.0,0.260960,0.250,0.222222,0.512122,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.531581
